In [1]:
!pip install pycountry


     |████████████████████████████████| 10.1 MB 5.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=10681845 sha256=0fcc1dab685dd474722f197fc839d0f02d2645d434a05b5e09f22ec63ce7c49d
  Stored in directory: /root/.cache/pip/wheels/0e/06/e8/7ee176e95ea9a8a8c3b3afcb1869f20adbd42413d4611c6eb4
Successfully built pycountry


In [2]:
import requests
from geopy.geocoders import Nominatim
import re
import io
import pandas as pd
from google.colab import drive
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
import json
import plotly.express as px
import pycountry

In [3]:
#Dictionary to map US states to State Codes
us_state_to_abbrev = {
"Alabama": "AL",
"Alaska": "AK",
"Arizona": "AZ",
"Arkansas": "AR",
"California": "CA",
"Colorado": "CO",
"Connecticut": "CT",
"Delaware": "DE",
"Florida": "FL",
"Georgia": "GA",
"Hawaii": "HI",
"Idaho": "ID",
"Illinois": "IL",
"Indiana": "IN",
"Iowa": "IA",
"Kansas": "KS",
"Kentucky": "KY",
"Louisiana": "LA",
"Maine": "ME",
"Maryland": "MD",
"Massachusetts": "MA",
"Michigan": "MI",
"Minnesota": "MN",
"Mississippi": "MS",
"Missouri": "MO",
"Montana": "MT",
"Nebraska": "NE",
"Nevada": "NV",
"New Hampshire": "NH",
"New Jersey": "NJ",
"New Mexico": "NM",
"New York": "NY",
"North Carolina": "NC",
"North Dakota": "ND",
"Ohio": "OH",
"Oklahoma": "OK",
"Oregon": "OR",
"Pennsylvania": "PA",
"Rhode Island": "RI",
"South Carolina": "SC",
"South Dakota": "SD",
"Tennessee": "TN",
"Texas": "TX",
"Utah": "UT",
"Vermont": "VT",
"Virginia": "VA",
"Washington": "WA",
"West Virginia": "WV",
"Wisconsin": "WI",
"Wyoming": "WY",
"District of Columbia": "DC",
"American Samoa": "AS",
"Guam": "GU",
"Northern Mariana Islands": "MP",
"Puerto Rico": "PR",
"United States Minor Outlying Islands": "UM",
"U.S. Virgin Islands": "VI",
}

In [4]:
from google.colab import auth
auth.authenticate_user()
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

In [5]:
from xlrd import open_workbook

#Authorize access to google drive and fetch the lat-lon file into a dataframe
drive.mount('/content/drive', force_remount=True)
dfll = pd.read_excel('/content/drive/My Drive/ColabNotebooks/inputFolder/lat-lon.xlsx')

#Add two new columns "City Name" and "State Code/Country Name"
dfll["City Name"] = pd.NaT
dfll["State Code/Country Name"] = pd.NaT
geolocator = Nominatim(user_agent='myapplication')
for index, row in dfll.iterrows():
  #Get city and state code (for US cities)/ country name using latitude and longitude
  address = geolocator.reverse(str(row[0])+","+str(row[1])).raw['address']
  dfll.loc[index,"City Name"]=address.get('city', '')
  if address.get('country', '')== "United States":
    dfll.loc[index,"State Code/Country Name"]= us_state_to_abbrev.get(address.get('state', ''))
  else:
    dfll.loc[index,"State Code/Country Name"]= address.get('country', '')

#Drop NaT values and update the lat-lon file
dfll = dfll.dropna()
#dfll.update([dfll.columns.values.tolist()] + dfll.values.tolist())
dfll.to_excel('/content/drive/My Drive/ColabNotebooks/outputFolder/lat-lon.xlsx',index=False)

from googleapiclient.http import MediaFileUpload
file_metadata = {
  'name': 'Sample file',
  'mimeType': 'text/plain'
}
media = MediaFileUpload('/content/drive/My Drive/ColabNotebooks/outputFolder/lat-lon.xlsx', 
                        mimetype='text/plain',
                        resumable=True)
created = drive_service.files().create(body=file_metadata,
                                       media_body=media,
                                       fields='id').execute()
print('File ID: {}'.format(created.get('id')))

Mounted at /content/drive
File ID: 1diQg_e6Pgcb7SxoAzDOsnS0V7Jf35bgr


In [6]:
#fetch the city-name file into a dataframe
dfcity = pd.read_excel('/content/drive/My Drive/ColabNotebooks/inputFolder/input.xlsx')

#Add two new columns "Latitude" and "Longitude"
dfcity["Latitude"] = pd.NaT
dfcity["Longitude"] = pd.NaT
geolocator = Nominatim(user_agent='myapplication')
for index, row in dfcity.iterrows():
  #Get latitude and longitude using city and state code (for US cities)/ country name 
  location = geolocator.geocode(row[0]+","+row[1],timeout=1000)
  if not location == None:
    dfcity.loc[index,"Latitude"]=location.latitude
    dfcity.loc[index,"Longitude"]=location.longitude

#Drop NaT values and update the lat-lon file
dfcity=dfcity.dropna()
dfcity.update([dfcity.columns.values.tolist()] + dfcity.values.tolist())





In [7]:
#Concatenate lat-lon and city-name dataframes without duplicates
df = pd.concat([dfcity,dfll], axis = 0).drop_duplicates().reset_index(drop=True)

In [8]:
#Add four new columns "Irradiance", "Avg temp", "Max temp", and "Min temp"
df["Irradiance"] = pd.NaT
df["Avg temp"]= pd.NaT
df["Max temp"]= pd.NaT
df["Min temp"]= pd.NaT
for index,row in df.iterrows():
  #Get irradiance and temperature for all the latitude and longitude values in the dataframe
  json_data = json.loads(requests.get('https://power.larc.nasa.gov/api/temporal/monthly/point?start=2016&end=2020&latitude='+str(row[2])+'&longitude='+str(row[3])+'&community=re&parameters=ALLSKY_SFC_SW_DWN%2CT2M%2CT2M_MAX%2CT2M_MIN&format=json&header=true').text)
  irradiance = json_data['properties']['parameter']['ALLSKY_SFC_SW_DWN']
  AvgTemp = json_data['properties']['parameter']['T2M']
  maxTemp = json_data['properties']['parameter']['T2M_MAX']
  minTemp = json_data['properties']['parameter']['T2M_MIN']
  df.loc[index,"Irradiance"] = (irradiance['201613']+irradiance['201713']+irradiance['201813']+irradiance['201913']+irradiance['202013'])/5
  df.loc[index,"Avg temp"] = (AvgTemp['201613']+AvgTemp['201713']+AvgTemp['201813']+AvgTemp['201913']+AvgTemp['202013'])/5
  df.loc[index,"Max temp"] = (maxTemp['201613']+maxTemp['201713']+maxTemp['201813']+maxTemp['201913']+maxTemp['202013'])/5
  df.loc[index,"Min temp"] = (minTemp['201613']+minTemp['201713']+minTemp['201813']+minTemp['201913']+minTemp['202013'])/5

In [9]:
import datetime
import time
current_time=datetime.datetime.now()

file_time = current_time.strftime("%y%m%d-%H%M%S")
df=df.dropna()
filename="irradiance-temp-"+file_time
df.to_excel('/content/drive/My Drive/ColabNotebooks/outputFolder/'+filename+'.xlsx',index=False)

from googleapiclient.http import MediaFileUpload
file_metadata = {
  'name': 'Sample file',
  'mimeType': 'text/plain'
}

media = MediaFileUpload('/content/drive/My Drive/ColabNotebooks/outputFolder/'+filename+'.xlsx', 
                        mimetype='text/plain',
                        resumable=True)
created = drive_service.files().create(body=file_metadata,
                                       media_body=media,
                                       fields='id').execute()


df.to_excel('/content/drive/My Drive/ColabNotebooks/outputFolder/irradiance-temp.xlsx',index=False)

file_metadata1 = {
  'name': 'Sample file',
  'mimeType': 'text/plain'
}

media = MediaFileUpload('/content/drive/My Drive/ColabNotebooks/outputFolder/irradiance-temp.xlsx', 
                        mimetype='text/plain',
                        resumable=True)
created = drive_service.files().create(body=file_metadata1,
                                       media_body=media,
                                       fields='id').execute()

File ID: 1gz9emjGiDWTvg1RSGfzAlIgFTVCmxXCq
